# Analyse der Sonnenstunden und Wetterstationen in der Schweiz (letzte 10 Jahre)

Dieses Notebook analysiert die Sonnenstunden in der Schweiz und visualisiert die wichtigsten Ergebnisse. Die Daten werden geladen, bereinigt, angereichert und in verschiedenen Diagrammen dargestellt.

## Daten laden, bereinigen und vorbereiten

In diesem Abschnitt werden die Rohdaten eingelesen, bereinigt und für die weitere Analyse vorbereitet.

In [ ]:
# Bibliotheken importieren
import pandas as pd
import geopandas as gpd
from matplotlib import cm, colors
import plotly.express as px
import plotly.graph_objects as go
import folium
import os
import locale
import re

# Dateipfade für die CSV-Dateien definieren
file_path_sun_data = '../data/meteo/meteo_swiss_data.csv'
file_path_stn_data = '../data/meteo/meteo_swiss_stn_data.csv'

# Wetterdaten und Stationsdaten laden
sun_data = pd.read_csv(file_path_sun_data, delimiter=';')
stn_data = pd.read_csv(file_path_stn_data, delimiter=';')

# Stationen aus Liechtenstein entfernen
stn_data = stn_data[stn_data['canton'] != 'LI']

def gms_to_dg(gms):
    """
    Wandelt eine Koordinate im Grad-Minuten-Format (z.B. '12°34') in Dezimalgrad um.
    """
    match = re.match(r"(\d+)°(\d+)", gms)
    if match:
        grad = int(match.group(1))
        minuten = int(match.group(2))
        return grad + (minuten / 60)
    raise ValueError("Ungültiges Format. Erwartet: 'XX°YY'")

# Koordinaten umrechnen
stn_data['latitude_dg'] = stn_data['latitude'].apply(gms_to_dg)
stn_data['longitude_dg'] = stn_data['longitude'].apply(gms_to_dg)

# Wetterdaten mit Stationsdaten verknüpfen
data = pd.merge(sun_data, stn_data, on='stn', how='inner')

# Zeitspalte in Datetime-Format umwandeln und Jahr/Monat extrahieren
data['time'] = pd.to_datetime(data['time'], format='%Y%m')
data['year'] = data['time'].dt.year
data['month'] = data['time'].dt.month

# Sonnenstunden bereinigen und in Float umwandeln
data['sunhours'] = data['su2000m0'].replace('-', '0').astype(float)

# Ungültige Werte entfernen
data = data[(data['sunhours'] > 0) & (data['year'] != 2025)]

# Durchschnittliche Sonnenstunden pro Monat und Jahr berechnen
monthly_data = data.groupby(['year', 'month'])['sunhours'].mean().round(0).unstack(level=0)

# Monatsnamen für Diagramme definieren
month_names = ['Jan', 'Feb', 'Mär', 'Apr', 'Mai', 'Jun', 'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dez']

def format_swiss(value, decimals=2):
    """
    Formatiert eine Zahl im Schweizer Zahlenformat mit gewünschter Dezimalstellenanzahl.
    """
    if isinstance(value, (int, float)):
        format_string = f"%.{decimals}f"
        return locale.format_string(format_string, value, grouping=True).replace(',', "'")
    return value

## Karte der Wetterstationen in der Schweiz

Die folgende Karte zeigt die geografische Verteilung der Wetterstationen.

In [ ]:
# Interaktive Karte der Wetterstationen erstellen
map = folium.Map(location=[46.8, 8.33], zoom_start=7, tiles='TopPlusOpen.Color')

# Kantonsgrenzen hinzufügen
folium.GeoJson('../data/meteo/swiss_cantons.geojson', name='Kantone').add_to(map)

# Marker für jede Wetterstation mit Popup-Informationen
for _, stn in stn_data.iterrows():
    popup_content = (
        f"<div style='width: 130px;'><b>Station:</b> {stn['stn']}<br>"
        f"<b>Name:</b> {stn['name']}<br>"
        f"<b>Höhe:</b> {stn['elevation']} m<br>"
        f"<b>Kanton:</b> {stn['canton_name']}</div>"
    )
    folium.Marker(
        location=[stn['latitude_dg'], stn['longitude_dg']],
        popup=popup_content
    ).add_to(map)

# Titel zur Karte hinzufügen
title_html = '''
<div style="font-size:20px;position: absolute;z-index: 1000;left: 25%;"><b>Verteilung der Wetterstationen in der Schweiz</b></div>
'''
map.get_root().html.add_child(folium.Element(title_html))

# Karte speichern und anzeigen
map.save('../docs/assets/diagramme/swiss_stations_map.html')
map

## Höhenverteilung der Wetterstationen

Die folgende Tabelle zeigt die Verteilung der Wetterstationen nach Höhenlage sowie die kumulierten Sonnenstunden pro Höhenkategorie und Jahr.

In [ ]:
# Höhenkategorien definieren
bins = [0, 500, 1000, 1500, 2000, 2500, 5000]
labels = ['<500 m', '500–1000 m', '1000–1500 m', '1500–2000 m', '2000–2500 m', '>2500 m']

# Höhenkategorie zuweisen
data['elevation_category'] = pd.cut(data['elevation'], bins=bins, labels=labels, right=False)
stn_data['elevation_category'] = pd.cut(stn_data['elevation'], bins=bins, labels=labels, right=False)

# Anzahl Stationen pro Höhenkategorie
stn_nr = stn_data.groupby('elevation_category', observed=True)['stn'].nunique().sort_index()

# Durchschnittliche Sonnenstunden pro Höhenkategorie, Jahr und Monat berechnen
elevation_monthly_avg = data.groupby(['elevation_category', 'year', 'month'], observed=True)['sunhours'].mean().reset_index()

# Kumulierte Sonnenstunden pro Höhenkategorie und Jahr berechnen
elevation_yearly_cum = elevation_monthly_avg.groupby(['elevation_category', 'year'], observed=True, dropna=False)['sunhours'].sum().reset_index()

# Pivot-Tabelle für die kumulierten Werte
elevation_yearly_pivot = elevation_yearly_cum.pivot(index='elevation_category', columns='year', values='sunhours')

# Markdown-Tabelle für die kumulierten Werte erstellen
markdown_table_cum = "| Höhenkategorie | Anzahl Stationen | " + " | ".join(str(year) for year in elevation_yearly_pivot.columns) + " |\n"
markdown_table_cum += "|:----------------:|:------------------:| " + " | ".join([":------------------:"] * len(elevation_yearly_pivot.columns)) + " |\n"

for category, row in elevation_yearly_pivot.iterrows():
    station_count = stn_nr.get(category, 0)
    markdown_table_cum += f"| {category} | {format_swiss(station_count)} | " + " | ".join(
        f"**{format_swiss(value, 0)}**" if year in row.nlargest(3).index else f"{format_swiss(value, 0)}"
        if not pd.isna(value) else "-" for year, value in row.items()
    ) + " |\n"

# Existierende Datei löschen, falls vorhanden
file_path_cum = "../docs/assets/md/hoehenverteilung_stationen.md"
if os.path.exists(file_path_cum):
    os.remove(file_path_cum)

# Tabelle als Markdown-Datei speichern
with open(file_path_cum, "w", encoding="utf-8") as f:
    f.write(markdown_table_cum)

print("Markdown-Datei wurde erfolgreich erstellt: hoehenverteilung_stationen.md")

## Durchschnittliche Sonnenstunden pro Monat

Das folgende Diagramm zeigt die durchschnittlichen Sonnenstunden pro Monat in der Schweiz über die letzten Jahre.

In [ ]:
# Durchschnittliche und Median-Sonnenstunden pro Monat berechnen
average_sunhours = monthly_data.mean(axis=1).round(0)
median_sunhours = monthly_data.median(axis=1).round(0)

# Farbpalette für die Jahre
year_colors = px.colors.qualitative.Plotly
color_map = {year: year_colors[i % len(year_colors)] for i, year in enumerate(monthly_data.columns)}

# Interaktives Liniendiagramm mit Plotly erstellen
fig = go.Figure()

# Linien für jedes Jahr hinzufügen
for year in monthly_data.columns:
    fig.add_trace(go.Scatter(
        x=monthly_data.index,
        y=monthly_data[year],
        mode='lines+markers',
        name=str(year),
        opacity=0.7,
        line=dict(dash='solid', color=color_map[year])
    ))

# Durchschnittslinie hinzufügen
fig.add_trace(go.Scatter(
    x=monthly_data.index,
    y=average_sunhours,
    mode='lines',
    name='Durchschnitt',
    line=dict(color='black', dash='solid', width=3)
))

# Medianlinie hinzufügen
fig.add_trace(go.Scatter(
    x=monthly_data.index,
    y=median_sunhours,
    mode='lines',
    name='Median',
    line=dict(color='red', dash='dot', width=2)
))

# Layout anpassen
fig.update_layout(
    title='Durchschnittliche Sonnenstunden pro Monat in der Schweiz',
    xaxis_title='Monat',
    yaxis_title='Durchschnittliche Sonnenstunden',
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(1, 13)),
        ticktext=month_names,
        showgrid=True,
        zeroline=True
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True
    ),
    legend_title='Legende',
    template='plotly_white',
    font=dict(size=12),
    margin=dict(l=40, r=40, t=40, b=40)
)

# Diagramm anzeigen und speichern
fig.show()
fig.write_html('../docs/assets/diagramme/sunhours_per_month.html')

## Durchschnittliche jährliche Sonnenstunden (Balkendiagramm)

Das folgende Diagramm zeigt für jedes Jahr die durchschnittlichen Sonnenstunden in der Schweiz. Der Mittelwert und Median über alle Jahre sind als Linien dargestellt.

In [ ]:
import plotly.graph_objects as go

# Durchschnittliche Sonnenstunden pro Jahr berechnen
avg_per_year = monthly_data.mean(axis=0)
median_per_year = monthly_data.median(axis=0)

# Gesamtdurchschnitt und Median berechnen
overall_avg = avg_per_year.mean()
overall_median = avg_per_year.median()

# Balkendiagramm erstellen
fig_bar = go.Figure()

# Balken für jedes Jahr
fig_bar.add_trace(go.Bar(
    x=avg_per_year.index.astype(str),
    y=avg_per_year.values,
    name='Jahresdurchschnitt',
    marker_color='#636EFA'
))

# Durchschnittslinie hinzufügen
fig_bar.add_trace(go.Scatter(
    x=avg_per_year.index.astype(str),
    y=[overall_avg]*len(avg_per_year),
    mode='lines',
    name='Durchschnitt (alle Jahre)',
    line=dict(color='black', dash='solid', width=3)
))

# Medianlinie hinzufügen
fig_bar.add_trace(go.Scatter(
    x=avg_per_year.index.astype(str),
    y=[overall_median]*len(avg_per_year),
    mode='lines',
    name='Median (alle Jahre)',
    line=dict(color='red', dash='dot', width=2)
))

fig_bar.update_layout(
    title='Durchschnittliche jährliche Sonnenstunden in der Schweiz',
    xaxis_title='',
    yaxis_title='Durchschnittliche Sonnenstunden',
    template='plotly_white',
    legend_title='Legende',
    font=dict(size=12),
    margin=dict(l=40, r=40, t=40, b=40)
)

fig_bar.show()
fig_bar.write_html('../docs/assets/diagramme/sunhours_per_year_bar.html')

## Verteilung der Sonnenstunden pro Monat (Boxplot)

Das folgende Diagramm zeigt die Verteilung der Sonnenstunden pro Monat in der Schweiz als Boxplot.

In [ ]:
# Interaktiven Boxplot der Sonnenstunden pro Monat erstellen
fig_box = px.box(
    data,
    x='month',
    y='sunhours',
    points=False,
    labels={'month': '', 'sunhours': 'Sonnenstunden'}
)

# Layout anpassen
fig_box.update_layout(
    title='Verteilung der Sonnenstunden pro Monat in der Schweiz',
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(1, 13)),
        ticktext=month_names
    ),
    template='plotly_white'
)

# Diagramm anzeigen und speichern
fig_box.show()
fig_box.write_html('../docs/assets/diagramme/sunhours_distribution_per_month.html')

## Verteilung der Sonnenstunden pro Monat (Violinplot)

Das folgende Diagramm zeigt die Verteilung der Sonnenstunden pro Monat in der Schweiz als Violinplot.

In [ ]:
import plotly.express as px

# Violinplot der Sonnenstunden pro Monat mit Farbpalette erstellen
fig_violin = px.violin(
    data,
    x='month',
    y='sunhours',
    box=True,
    labels={'month': '', 'sunhours': 'Sonnenstunden'},
    title='Violinplot der Verteilung der Sonnenstunden pro Monat in der Schweiz',
    color_discrete_sequence=px.colors.sequential.Sunset
)

# Layout anpassen
fig_violin.update_layout(
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(1, 13)),
        ticktext=month_names
    ),
    template='plotly_white'
)

# Diagramm anzeigen und speichern
fig_violin.show()
fig_violin.write_html('../docs/assets/diagramme/sunhours_violinplot_per_month.html')

## Durchschnittliche jährliche Sonnenstunden pro Kanton (Karte)

Die folgende Karte zeigt die durchschnittlichen jährlichen Sonnenstunden pro Kanton.

In [ ]:
# Durchschnittliche Sonnenstunden pro Kanton berechnen
canton_monthly_avg = data.groupby(['canton_name', 'year', 'month'])['sunhours'].mean().reset_index()
canton_yearly_cum = canton_monthly_avg.groupby(['canton_name', 'year'])['sunhours'].sum().reset_index()
canton_avg = canton_yearly_cum.groupby('canton_name')['sunhours'].mean().reset_index()

# GeoJSON-Daten der Schweizer Kantone laden
cantons = gpd.read_file('../data/meteo/swiss_cantons.geojson')

# Durchschnittswerte mit GeoJSON-Daten verknüpfen
cantons = cantons.merge(canton_avg, left_on='NAME', right_on='canton_name', how='left')

# Fehlende Werte mit Gesamtdurchschnitt befüllen
overall_avg_sunhours = canton_avg['sunhours'].mean()
cantons['sunhours'] = cantons['sunhours'].fillna(overall_avg_sunhours)

# Interaktive Karte mit Folium erstellen
map = folium.Map(location=[46.8, 8.33], zoom_start=7)

# Kantonsgrenzen hinzufügen
folium.GeoJson(
    '../data/meteo/swiss_cantons.geojson', 
    name='Kantone'
).add_to(map)

# Farbskala für die Kreise definieren
norm = colors.Normalize(vmin=canton_avg['sunhours'].min(), vmax=canton_avg['sunhours'].max())
colormap = cm.ScalarMappable(norm=norm, cmap='YlOrRd')

# Kreise für die Durchschnittswerte der Sonnenstunden hinzufügen
for _, row in cantons.iterrows():
    if not pd.isna(row['sunhours']):
        color = colors.to_hex(colormap.to_rgba(row['sunhours']))
        folium.Circle(
            location=[row['geometry'].centroid.y, row['geometry'].centroid.x],
            radius=row['sunhours'] * 7,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.6,
            popup=f"{row['NAME']}: {row['sunhours']:.2f}"
        ).add_to(map)

# Textmarker mit den Durchschnittswerten hinzufügen
for _, row in cantons.iterrows():
    if not pd.isna(row['sunhours']):
        folium.map.Marker(
            [row['geometry'].centroid.y, row['geometry'].centroid.x],
            icon=folium.DivIcon(
                icon_size=(15, 12),
                icon_anchor=(20, 12),
                html=f'<div style="font-size: 12pt; color: black;">{int(row["sunhours"])}</div>'
            ),
            popup=f"{row['NAME']}: {row['sunhours']:.2f}"
        ).add_to(map)

# Titel zur Karte hinzufügen
title_html = '''
<div style="font-size:20px;position: absolute;z-index: 1000;left: 25%;"><b>Durchschnittliche jährliche Sonnenstunden pro Kanton</b></div>
'''
map.get_root().html.add_child(folium.Element(title_html))

# Karte speichern und anzeigen
map.save('../docs/assets/diagramme/swiss_sunhours_map.html')
map

## Kumulierte Sonnenstunden pro Jahr und Station (3D-Animation)

Das folgende 3D-Diagramm zeigt die kumulierten Sonnenstunden pro Jahr und Station in der Schweiz.

In [ ]:
# Kumulierte Sonnenstunden pro Station und Jahr berechnen
data_grouped = data.groupby(["year", "stn", "name", "latitude_dg", "longitude_dg"]).agg({"sunhours": "sum"}).reset_index()
data_grouped["sunhours_cum"] = data_grouped.groupby("stn")["sunhours"].cumsum()
data_grouped["name2"] = data_grouped["name"]

# Liste der Jahre sortieren
years = sorted(data_grouped["year"].unique())

def get_color(value):
    """
    Gibt eine Farbe abhängig von der Höhe der kumulierten Sonnenstunden zurück.
    """
    if value <= 2500: return "rgb(0, 0, 139)"
    elif value <= 5000: return "rgb(30, 144, 255)"
    elif value <= 7500: return "rgb(60, 179, 113)"
    elif value <= 10000: return "rgb(124, 252, 0)"
    elif value <= 12500: return "rgb(255, 255, 0)"
    elif value <= 15000: return "rgb(255, 165, 0)"
    elif value <= 17500: return "rgb(255, 69, 0)"
    elif value <= 20000: return "rgb(220, 20, 60)"
    else: return "rgb(139, 0, 0)"

def get_line_width(value):
    """
    Gibt die Linienbreite abhängig von der Höhe der kumulierten Sonnenstunden zurück.
    """
    if value < 5000: return 2
    elif value < 10000: return 3
    elif value < 15000: return 4
    elif value < 20000: return 5
    else: return 6

# Start-Frame für das erste Jahr erstellen
df_start = data_grouped[data_grouped["year"] == years[0]]

# Start-Traces für alle Stationen im ersten Jahr
start_traces = [
    go.Scatter3d(
        x=[row["longitude_dg"]]*2,
        y=[row["latitude_dg"]]*2,
        z=[0, row["sunhours_cum"]],
        mode="lines",
        line=dict(
            color=get_color(row["sunhours_cum"]),
            width=get_line_width(row["sunhours_cum"])
        ),
        text=f"Station: {row.stn}<br>Name: {row.name2}<br>Jahr: {years[0]}<br>Sonnenstunden: {int(row.sunhours_cum):,}".replace(',', ''),
        hoverinfo="text",
        showlegend=False
    ) for _, row in df_start.iterrows()
]

# GeoJSON-Daten für die Schweizerkarte laden
swiss_map = gpd.read_file('../data/meteo/swiss_cantons.geojson')

# Koordinaten für die Kantonsgrenzen extrahieren
map_lines = []
for _, row in swiss_map.iterrows():
    if row['geometry'].geom_type == 'Polygon':
        x, y = row['geometry'].exterior.xy
        map_lines.append(go.Scatter3d(
            x=x.tolist(),
            y=y.tolist(),
            z=[0] * len(x),
            mode='lines',
            line=dict(color='black', width=2),
            name=row['NAME'],
            showlegend=False
        ))
    elif row['geometry'].geom_type == 'MultiPolygon':
        for polygon in row['geometry']:
            x, y = polygon.exterior.xy
            map_lines.append(go.Scatter3d(
                x=x,
                y=y,
                z=[0] * len(x),
                mode='lines',
                line=dict(color='black', width=2),
                name=row['NAME'],
                showlegend=False
            ))

# Schweizerkarte als Basis zu den Start-Traces hinzufügen
start_traces.extend(map_lines)

# Animations-Frames für jedes Jahr erstellen
frames = [
    go.Frame(
        data=[
            go.Scatter3d(
                x=[row["longitude_dg"]]*2,
                y=[row["latitude_dg"]]*2,
                z=[0, row["sunhours_cum"]],
                mode="lines",
                line=dict(
                    color=get_color(row["sunhours_cum"]),
                    width=get_line_width(row["sunhours_cum"])
                ),
                text=f"Station: {row.stn}<br>Name: {row.name2}<br>Jahr: {jahr}<br>Sonnenstunden: {int(row.sunhours_cum):,}".replace(',', ''),
                hoverinfo="text",
                showlegend=False
            ) for _, row in data_grouped[data_grouped["year"] == jahr].head(110).iterrows()
        ] + map_lines,
        name=str(jahr)
    ) for jahr in years
]

# Layout für das 3D-Diagramm definieren
layout = go.Layout(
    title="☀️ Kumulierte Sonnenstunden in der Schweiz der letzten 10 Jahre",
    scene=dict(
        xaxis=dict(
            title="Längengrad",
            showgrid=True,
            backgroundcolor="rgba(0, 0, 0,0)",
            gridcolor="white",
            showbackground=True,
            zerolinecolor="white"),
        yaxis=dict(
            title="Breitengrad",
            backgroundcolor="rgba(0, 0, 0,0)",
            gridcolor="white",
            showbackground=True,
            zerolinecolor="white"),
        zaxis=dict(
            title="Kumulierte Sonnenstunden",
            tickvals=[5000, 10000, 15000, 20000],
            ticktext=["5'000", "10'000", "15'000", "20'000"],
            range=[0, data_grouped["sunhours_cum"].max()],
            backgroundcolor="rgba(0, 0, 0,0)",
            gridcolor="white",
            showbackground=True,
            zerolinecolor="white"
        ),
        camera=dict(
            eye=dict(x=0.5, y=-2.5, z=1.5)
        )
    ),
    sliders=[{
        "steps": [{
            "method": "animate",
            "label": str(j),
            "args": [[str(j)], {"mode": "immediate", "frame": {"duration": 600, "redraw": True}}]
        } for j in years]
    }],
    updatemenus=[{
        "type": "buttons",
        "buttons": [
            {"label": "Play", "method": "animate", "args": [None]},
            {"label": "Pause", "method": "animate", "args": [[None], {"mode": "immediate", "frame": {"duration": 0}}]}
        ]
    }]
)

# 3D-Diagramm erstellen, speichern und anzeigen
fig = go.Figure(data=start_traces, layout=layout, frames=frames)
fig.write_html("../docs/assets/diagramme/sonnenstunden_3d.html")
fig.show()